In [ ]:
pip install ipywebrtc

In [ ]:
"""import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('model.h5')

# Define the labels for the classes that the model can predict
labels = ['Real','Fake']

# Start capturing video from the default camera
cap = cv2.VideoCapture(0)

# Loop through frames in the video feed
while True:
    # Read a single frame from the video feed
    ret, frame = cap.read()

    # Convert the frame to a format that the model can use for prediction
    frame = cv2.resize(frame, (64, 64))
    frame = np.expand_dims(frame, axis=0)

    # Make a prediction using the pre-trained model
    pred = model.predict(frame)

    # Get the index of the predicted class
    pred_idx = np.argmax(pred)

    # Get the label for the predicted class
    pred_label = labels[pred_idx]

    # Display the label on the video feed
    cv2.putText(frame, pred_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the video feed with the predicted label
    cv2.imshow('frame', frame)

    # Break out of the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()"""

import ipywebrtc as webrtc
import PIL.Image
import numpy as np
import cv2
import base64
from keras.models import load_model

# Load the pre-trained model
model = load_model('model.h5')

# Define the labels for the classes that the model can predict
labels = ['Real', 'Fake']

# Define a function to handle each new video frame
def handle_frame(frame):
    # Decode the JPEG image data from base64
    jpg_data = base64.b64decode(frame.data.split(',')[1])

    # Convert the JPEG image data to a numpy array
    np_frame = np.frombuffer(jpg_data, np.uint8)
    np_frame = cv2.imdecode(np_frame, cv2.IMREAD_COLOR)

    # Convert the frame to a format that the model can use for prediction
    frame = cv2.resize(np_frame, (64, 64))
    frame = np.expand_dims(frame, axis=0)

    # Make a prediction using the pre-trained model
    pred = model.predict(frame)

    # Get the index of the predicted class
    pred_idx = np.argmax(pred)

    # Get the label for the predicted class
    pred_label = labels[pred_idx]

    # Display the label on the video feed
    cv2.putText(np_frame, pred_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Convert the numpy array to a PIL image
    pil_image = PIL.Image.fromarray(np_frame)

    # Show the PIL image in the notebook
    display(pil_image)

# Create a video recorder object
video_recorder = webrtc.VideoRecorder(stream=webrtc.CameraStream.facing_user())

# Register the frame handler function to be called for each new frame
video_recorder.observe(handle_frame, names='data')

# Display the video recorder widget
display(video_recorder)
